In [1]:
import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3

In [2]:
class conv_2_layer(Layer):
    def __init__(self,filters=32,kernel_size=2,padding="valid",name=None):
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.name = name
        
        
    def concat2(self,x,name=None):
        conv1 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv2 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv3 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        cat = concatenate([conv2,conv1])
        cat2 = concatenate([cat,conv3])

        conv_1 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv_2 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv_3 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        cat_1 = concatenate([conv_2,conv_1])
        cat_2 = concatenate([cat_1,conv_3])
        
        z = concatenate([cat2,cat_2])
        return z

In [3]:
class conv_3_layer(Layer):
    def __init__(self,filters=32,kernel_size=2,padding="valid",name=None):
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.name=name
        
    def concat3(self,x):
        conv1 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv2 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv3 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        cat = concatenate([conv2,conv1])
        
        cat2 = concatenate([cat,conv3])

        conv_1 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv_2 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv_3 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        cat_1 = concatenate([conv_2,conv_1])
        
        cat_2 = concatenate([cat_1,conv_3])
        
        conv4 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv5 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        conv6 = Conv1D(self.filters, self.kernel_size, self.padding, kernel_initializer="glorot_uniform")(x)
        cat3 = concatenate([conv4,conv5])
        
        cat4 = concatenate([cat3,conv6])
        
        l = concatenate([cat2,cat_2])
        z = concatenate([l,cat4])
        
        return z
        

In [4]:
import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [5]:
class squash_function(Layer):
    def squash(s, axis=-1, epsilon=1e-7, name=None):
        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keep_dims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector